# Decorators
Article: https://realpython.com/primer-on-python-decorators/

In [78]:
import numpy as np
from functools import lru_cache, wraps
from datetime import datetime
import pandas as pd

## Decorators provided by Python
- dataclass
- staticmethod, classmethod
- override

In [2]:
def fibo(n):
    assert n >= 0
    n1 = 0
    n2 = 1
    match n:
        case 1:
            return n1
        case 2:
            return n2
        case _:
            for _ in range(n-2):
                n1, n2 = n2, n1 + n2
            return n2

In [3]:
fibo(1), fibo(2), fibo(10)

(0, 1, 34)

In [4]:
[fibo(n) for n in range(1, 11)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [5]:
fibo(100)

218922995834555169026

In [6]:
%timeit -n 1000 [fibo(n) for n in range(1, 100)]

217 μs ± 23.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
@lru_cache
def fibo2(n):
    assert n >= 0
    n1 = 0
    n2 = 1
    match n:
        case 1:
            return n1
        case 2:
            return n2
        case _:
            for _ in range(n-2):
                n1, n2 = n2, n1 + n2
            return n2

In [8]:
%timeit -n 1  -r 1 [fibo2(n) for n in range(1, 100)]

215 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%timeit -n 1 -r 1 [fibo2(n) for n in range(1, 100)]

20.8 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
def fibo_rec(n):
    assert n >= 0
    n1 = 0
    n2 = 1
    match n:
        case 1:
            return n1
        case 2:
            return n2
        case _:
            return fibo_rec(n-2) + fibo_rec(n-1)

In [11]:
[fibo_rec(n) for n in range(1, 11)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [16]:
%timeit -n 10 -r 2 fibo_rec(35)

2.72 s ± 67 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [24]:
@lru_cache(1000)
def fibo_rec2(n):
    """Compute the nth term of the Fibonacci series"""
    assert n >= 0
    n1 = 0
    n2 = 1
    match n:
        case 1:
            return n1
        case 2:
            return n2
        case _:
            return fibo_rec2(n-2) + fibo_rec2(n-1)

In [25]:
%timeit -n 10 -r 2 fibo_rec2(35)

The slowest run took 17.06 times longer than the fastest. This could mean that an intermediate result is being cached.
3.07 μs ± 2.73 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [26]:
%timeit -n 10 -r 2 fibo_rec2(35)

300 ns ± 130 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [27]:
fibo_rec2(100)

218922995834555169026

In [28]:
fibo_rec2?

Signature:       fibo_rec2(n)
Call signature:  fibo_rec2(*args, **kwargs)
Type:            _lru_cache_wrapper
String form:     <functools._lru_cache_wrapper object at 0x000001E1944B6770>
File:            c:\users\matth\appdata\local\temp\ipykernel_35640\2719094636.py
Docstring:       Compute the nth term of the Fibonacci series
Class docstring:
Create a cached callable that wraps another function.

user_function:      the function being cached

maxsize:  0         for no caching
          None      for unlimited cache size
          n         for a bounded cache

typed:    False     cache f(3) and f(3.0) as identical calls
          True      cache f(3) and f(3.0) as distinct calls

cache_info_type:    namedtuple class with the fields:
                        hits misses currsize maxsize

In [29]:
# a Python decorator is a function
fibo3 = lru_cache(fibo)
fibo3?

Signature:       fibo3(n)
Call signature:  fibo3(*args, **kwargs)
Type:            _lru_cache_wrapper
String form:     <functools._lru_cache_wrapper object at 0x000001E1944B4A90>
File:            c:\users\matth\appdata\local\temp\ipykernel_35640\3863838992.py
Docstring:       <no docstring>
Class docstring:
Create a cached callable that wraps another function.

user_function:      the function being cached

maxsize:  0         for no caching
          None      for unlimited cache size
          n         for a bounded cache

typed:    False     cache f(3) and f(3.0) as identical calls
          True      cache f(3) and f(3.0) as distinct calls

cache_info_type:    namedtuple class with the fields:
                        hits misses currsize maxsize

In [30]:
fibo3(100)

218922995834555169026

## Faisons nos propes décorations

In [79]:
def logger(f):
    @wraps(f)
    def logger_wrapper(*args, **kwargs):
        dt1 = datetime.now()
        print(f'LOG [{dt1}]: positional args = {args}')
        print(f'LOG [{dt1}]: keyword args = {kwargs}')
        r = f(*args, **kwargs)
        dt2 = datetime.now()
        print(f'LOG [{dt2}]: results = {r}')
        return r
    return logger_wrapper

In [80]:
@logger
def compute(x: int) -> int:
    """apply formula x**2 + 1"""
    return x**2+1

In [81]:
res = compute(12)
res

LOG [2025-03-20 15:37:27.619627]: positional args = (12,)
LOG [2025-03-20 15:37:27.619627]: keyword args = {}
LOG [2025-03-20 15:37:27.619769]: results = 145


145

In [82]:
@logger
def do_anything() -> None:
    print("I'm doing what I want !")

In [83]:
do_anything()

LOG [2025-03-20 15:37:29.710347]: positional args = ()
LOG [2025-03-20 15:37:29.710347]: keyword args = {}
I'm doing what I want !
LOG [2025-03-20 15:37:29.710890]: results = None


In [84]:
normal = logger(np.random.normal)

In [85]:
x = normal(10.0, 2.5, 100_000)
x

LOG [2025-03-20 15:37:33.967117]: positional args = (10.0, 2.5, 100000)
LOG [2025-03-20 15:37:33.967117]: keyword args = {}
LOG [2025-03-20 15:37:33.972054]: results = [10.15404023  9.99536777 11.52993061 ... 10.46398437 11.70760173
  8.77421078]


array([10.15404023,  9.99536777, 11.52993061, ..., 10.46398437,
       11.70760173,  8.77421078], shape=(100000,))

In [86]:
fibo_log = logger(fibo_rec2)
fibo_log(n=100)

LOG [2025-03-20 15:37:34.578369]: positional args = ()
LOG [2025-03-20 15:37:34.578369]: keyword args = {'n': 100}
LOG [2025-03-20 15:37:34.578502]: results = 218922995834555169026


218922995834555169026

In [87]:
read_csv = logger(pd.read_csv)

In [88]:
df = read_csv('data/cities100k.csv', sep=';', encoding='cp1252')
df.head(2)

LOG [2025-03-20 15:37:35.646531]: positional args = ('data/cities100k.csv',)
LOG [2025-03-20 15:37:35.646531]: keyword args = {'sep': ';', 'encoding': 'cp1252'}
LOG [2025-03-20 15:37:35.652127]: results =     code_insee          nom_standard       nom_sans_pronom  \
0        75056                 Paris                 Paris   
1        13055             Marseille             Marseille   
2        69123                  Lyon                  Lyon   
3        31555              Toulouse              Toulouse   
4         6088                  Nice                  Nice   
5        44109                Nantes                Nantes   
6        34172           Montpellier           Montpellier   
7        67482            Strasbourg            Strasbourg   
8        33063              Bordeaux              Bordeaux   
9        59350                 Lille                 Lille   
10       35238                Rennes                Rennes   
11       83137                Toulon               

,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,reg_code,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,75056,Paris,Paris,à Paris,de Paris,paris,PARIS,COM,commune,11,...,2.352,NaN,NaN,1,Grands centres urbains,4.0,centres majeurs d'équipements et de services,Parisien,https://fr.wikipedia.org/wiki/fr:Paris,https://villedereve.fr/ville/75056-paris
1,13055,Marseille,Marseille,à Marseille,de Marseille,marseille,MARSEILLE,COM,commune,93,...,5.370,NaN,NaN,1,Grands centres urbains,4.0,centres majeurs d'équipements et de services,Marseillais,https://fr.wikipedia.org/wiki/fr:Marseille,https://villedereve.fr/ville/13055-marseille


In [89]:
read_csv?

Signature:
read_csv(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols: 'UsecolsArgType' = None,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters: 'Mapping[Hashable, Callable] | None' = None,
    true_values: 'list | None' = None,
    false_values: 'list | None' = None,
    skipinitialspace: 'bool' = False,
    skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values: 'Hashable | Iterable[Hashable] | Mapping[Hashable, Iterable[Hashable]] | None' = None,
    keep_default_na: 'bool' = True,
    na_filter: 'boo

In [90]:
read_csv.__name__

'read_csv'

## Plusieurs décorations

In [91]:
@logger
@lru_cache
def fibo_rec4(n):
    assert n >= 0
    n1 = 0
    n2 = 1
    match n:
        case 1:
            return n1
        case 2:
            return n2
        case _:
            return fibo_rec4(n-2) + fibo_rec4(n-1)

In [93]:
fibo_rec4(10)

LOG [2025-03-20 15:41:33.699876]: positional args = (10,)
LOG [2025-03-20 15:41:33.699876]: keyword args = {}
LOG [2025-03-20 15:41:33.700028]: positional args = (8,)
LOG [2025-03-20 15:41:33.700028]: keyword args = {}
LOG [2025-03-20 15:41:33.700069]: positional args = (6,)
LOG [2025-03-20 15:41:33.700069]: keyword args = {}
LOG [2025-03-20 15:41:33.700107]: positional args = (4,)
LOG [2025-03-20 15:41:33.700107]: keyword args = {}
LOG [2025-03-20 15:41:33.700144]: results = 2
LOG [2025-03-20 15:41:33.700164]: positional args = (5,)
LOG [2025-03-20 15:41:33.700164]: keyword args = {}
LOG [2025-03-20 15:41:33.700198]: results = 3
LOG [2025-03-20 15:41:33.700218]: results = 5
LOG [2025-03-20 15:41:33.700238]: positional args = (7,)
LOG [2025-03-20 15:41:33.700238]: keyword args = {}
LOG [2025-03-20 15:41:33.700273]: positional args = (5,)
LOG [2025-03-20 15:41:33.700273]: keyword args = {}
LOG [2025-03-20 15:41:33.700308]: results = 3
LOG [2025-03-20 15:41:33.700326]: positional args = 

34

In [94]:
@lru_cache
@logger
@lru_cache
@logger
def fibo_rec5(n):
    assert n >= 0
    n1 = 0
    n2 = 1
    match n:
        case 1:
            return n1
        case 2:
            return n2
        case _:
            return fibo_rec5(n-2) + fibo_rec5(n-1)

In [95]:
fibo_rec5(10)

LOG [2025-03-20 15:42:53.101848]: positional args = (10,)
LOG [2025-03-20 15:42:53.101848]: keyword args = {}
LOG [2025-03-20 15:42:53.102282]: positional args = (10,)
LOG [2025-03-20 15:42:53.102282]: keyword args = {}
LOG [2025-03-20 15:42:53.102331]: positional args = (8,)
LOG [2025-03-20 15:42:53.102331]: keyword args = {}
LOG [2025-03-20 15:42:53.102370]: positional args = (8,)
LOG [2025-03-20 15:42:53.102370]: keyword args = {}
LOG [2025-03-20 15:42:53.102411]: positional args = (6,)
LOG [2025-03-20 15:42:53.102411]: keyword args = {}
LOG [2025-03-20 15:42:53.102447]: positional args = (6,)
LOG [2025-03-20 15:42:53.102447]: keyword args = {}
LOG [2025-03-20 15:42:53.102486]: positional args = (4,)
LOG [2025-03-20 15:42:53.102486]: keyword args = {}
LOG [2025-03-20 15:42:53.102522]: positional args = (4,)
LOG [2025-03-20 15:42:53.102522]: keyword args = {}
LOG [2025-03-20 15:42:53.102559]: positional args = (2,)
LOG [2025-03-20 15:42:53.102559]: keyword args = {}
LOG [2025-03-20 1

34

In [98]:
@logger
def fibo6(n):
    @lru_cache
    def fibo_rec6(n):
        assert n >= 0
        n1 = 0
        n2 = 1
        match n:
            case 1:
                return n1
            case 2:
                return n2
            case _:
                return fibo_rec6(n-2) + fibo_rec6(n-1)
    return fibo_rec6(n)

In [99]:
fibo6(100)

LOG [2025-03-20 15:45:55.391994]: positional args = (100,)
LOG [2025-03-20 15:45:55.391994]: keyword args = {}
LOG [2025-03-20 15:45:55.393012]: results = 218922995834555169026


218922995834555169026